In [1]:
import zipfile
from pathlib import Path

zip_path = Path("data/raw/cricsheet/odis_yaml.zip")
extracted_path = Path("data/raw/cricsheet/odis_yaml")

extracted_path.mkdir(parents=True,exist_ok=True)

with zipfile.ZipFile(zip_path,'r') as zip_ref:
    zip_ref.extractall(extracted_path)

print("Yaml Files Extracted.")

Yaml Files Extracted.


In [ ]:
### 2. Yaml -> Structured Ball-by Ball data

In [3]:
import yaml
import pandas as pd
from pathlib import Path
from tqdm import tqdm

yaml_dir = Path("data/raw/cricsheet/odis_yaml")
yaml_files = list(yaml_dir.glob("*.yaml"))

print(f"Total Matches Found: {len(yaml_files)}")

Total Matches Found: 2504


In [6]:
yaml_files[785]

WindowsPath('data/raw/cricsheet/odis_yaml/1384428.yaml')

In [10]:
rows =[]

for file in tqdm(yaml_files):

    with open(file,"r",encoding="utf-8") as f:
        match = yaml.safe_load(f)

    match_id = file.stem
    teams = match["info"]["teams"]

    for innings_no , innings in enumerate(match["innings"],start=1):
        innings_name = list(innings.keys())[0]
        innings_data = innings[innings_name]

        batting_team = innings_data["team"]
        bowling_team = [t for t in teams if t !=  batting_team][0]

        for delivery in innings_data["deliveries"]:
            ball_key = list(delivery.keys())[0]
            ball_data = delivery[ball_key]

            over = int(float(ball_key))
            ball = int(round((float(ball_key)-over)*10))

            runs = ball_data.get("runs",{})
            wicket = ball_data.get("wicket")

            wicket = ball_data.get("wicket")

            if isinstance(wicket, list):
                player_out = wicket[0].get("player_out")
                wicket_count = len(wicket)

            elif isinstance(wicket, dict):
                player_out = wicket.get("player_out")
                wicket_count = 1

            else:
                player_out = None
                wicket_count = 0


            
            rows.append({
                "match_id":match_id,
                "innings":innings_no,
                "batting_team": batting_team,
                "bowling_team": bowling_team,
                "over": over,
                "ball":ball,
                "batter":ball_data.get("batsman"),
                "non_striker":ball_data.get("non_striker"),
                "bowler":ball_data.get("bowler"),
                "runs_batter":runs.get("batsman",0),
                "runs_extras":runs.get("extras",0),
                "runs_total":runs.get("total",0),
                "wicket": wicket_count,
                "player_out": player_out
            })
            

100%|██████████| 2504/2504 [30:10<00:00,  1.38it/s] 


In [11]:
rows[:10]

[{'match_id': '1000887',
  'innings': 1,
  'batting_team': 'Australia',
  'bowling_team': 'Pakistan',
  'over': 0,
  'ball': 1,
  'batter': 'DA Warner',
  'non_striker': 'TM Head',
  'bowler': 'Mohammad Amir',
  'runs_batter': 0,
  'runs_extras': 0,
  'runs_total': 0,
  'wicket': 0,
  'player_out': None},
 {'match_id': '1000887',
  'innings': 1,
  'batting_team': 'Australia',
  'bowling_team': 'Pakistan',
  'over': 0,
  'ball': 2,
  'batter': 'DA Warner',
  'non_striker': 'TM Head',
  'bowler': 'Mohammad Amir',
  'runs_batter': 0,
  'runs_extras': 0,
  'runs_total': 0,
  'wicket': 0,
  'player_out': None},
 {'match_id': '1000887',
  'innings': 1,
  'batting_team': 'Australia',
  'bowling_team': 'Pakistan',
  'over': 0,
  'ball': 3,
  'batter': 'DA Warner',
  'non_striker': 'TM Head',
  'bowler': 'Mohammad Amir',
  'runs_batter': 0,
  'runs_extras': 0,
  'runs_total': 0,
  'wicket': 0,
  'player_out': None},
 {'match_id': '1000887',
  'innings': 1,
  'batting_team': 'Australia',
  'bowl

In [12]:
ball_df = pd.DataFrame(rows)
ball_df.head()

,match_id,innings,batting_team,bowling_team,over,ball,batter,non_striker,bowler,runs_batter,runs_extras,runs_total,wicket,player_out
0,1000887,1,Australia,Pakistan,0,1,DA Warner,TM Head,Mohammad Amir,0,0,0,0,None
1,1000887,1,Australia,Pakistan,0,2,DA Warner,TM Head,Mohammad Amir,0,0,0,0,None
2,1000887,1,Australia,Pakistan,0,3,DA Warner,TM Head,Mohammad Amir,0,0,0,0,None
3,1000887,1,Australia,Pakistan,0,4,DA Warner,TM Head,Mohammad Amir,0,0,0,0,None
4,1000887,1,Australia,Pakistan,0,5,DA Warner,TM Head,Mohammad Amir,0,1,1,0,None


In [13]:
Path("data/processed").mkdir(parents=True,exist_ok=True)
ball_df.to_csv("data/processed/ball_by_ball.csv",index=False)
print("ball_by_ball.csv created")

ball_by_ball.csv created


In [14]:
ball_df.shape

(1327866, 14)

In [15]:
ball_df["innings"].value_counts()

innings
1    721704
2    606096
3        34
4        32
Name: count, dtype: int64

In [16]:
ball_df["wicket"].sum()

np.int64(36299)

In [19]:
ball_df.isna().sum().head()

match_id        0
innings         0
batting_team    0
bowling_team    0
over            0
dtype: int64

### ball_by_ball.csv -> clean_ball_by_ball.csv -> innings_summary.csv -> match_summary.csv

In [6]:
import pandas as pd
from pathlib import Path
ball_df = pd.read_csv("data/processed/ball_by_ball.csv")
ball_df.shape

(1327866, 14)

In [7]:
clean_ball_df = ball_df[ball_df["innings"].isin([1,2])].copy()
clean_ball_df["innings"].value_counts()

innings
1    721704
2    606096
Name: count, dtype: int64

Sort chronologically (Critical) for seq to seq

In [8]:
clean_ball_df = clean_ball_df.sort_values(by=['match_id','innings','over','ball']).reset_index(drop=True)

In [24]:
clean_ball_df.to_csv("data/processed/clean_ball_by_ball.csv",index=False)
print("clean_ball_by_ball.csv saved")

clean_ball_by_ball.csv saved


Innings-Level Table

In [9]:
innings_df = (
    clean_ball_df.groupby(["match_id","innings","batting_team"])
    .agg(
        total_runs = ("runs_total","sum"),
        wickets = ("wicket","sum"),
        ball_faced = ('runs_total','count')
    )
    .reset_index()
    
    )
innings_df.head()


,match_id,innings,batting_team,total_runs,wickets,ball_faced
0,64814,1,New Zealand,254,9,309
1,64814,2,India,219,10,272
2,64815,1,India,108,10,261
3,64815,2,New Zealand,109,5,171
4,64816,1,India,122,10,278


In [10]:
innings_df.to_csv(
    "data/processed/innings_summary.csv",index=False
)
print("innings_summary.csv saved")

innings_summary.csv saved


### Create match Level Table

In [11]:
match_df = (
    innings_df.pivot(index="match_id",columns='innings',values=['batting_team','total_runs'])
)
match_df.columns =[
    "team_1","team_2",
    "team_1_runs","team_2_runs"
]
match_df = match_df.reset_index()
match_df.head()

,match_id,team_1,team_2,team_1_runs,team_2_runs
0,64814,New Zealand,India,254,219
1,64815,India,New Zealand,108,109
2,64816,India,New Zealand,122,123
3,64817,New Zealand,India,168,169
4,64819,India,New Zealand,122,125


### Decide winner

In [12]:
match_df["winner"] = match_df.apply(
    lambda x: x["team_1"] if x["team_1_runs"] > x["team_2_runs"]
    else x['team_2'],axis=1
)

In [13]:
match_df.head()

,match_id,team_1,team_2,team_1_runs,team_2_runs,winner
0,64814,New Zealand,India,254,219,New Zealand
1,64815,India,New Zealand,108,109,New Zealand
2,64816,India,New Zealand,122,123,New Zealand
3,64817,New Zealand,India,168,169,India
4,64819,India,New Zealand,122,125,New Zealand


In [14]:
match_df.to_csv(
    "data/processed/match_summary.csv",index=False
)
print("match_summary.csv saved")

match_summary.csv saved


In [16]:
print(innings_df.shape)
print(match_df.shape)

print(innings_df["innings"].value_counts())

print((match_df["team_1_runs"] >=0) .all())
print((match_df["team_2_runs"] >=0).all())

(4942, 6)
(2504, 6)
innings
1    2504
2    2438
Name: count, dtype: int64
True
False


In [22]:
neg_team2 = match_df[match_df["team_2_runs"] < 0]
neg_team2

,match_id,team_1,team_2,team_1_runs,team_2_runs,winner


In [23]:
match_df["team_2_runs"].isna().sum()


np.int64(66)

In [24]:
match_df[match_df["team_2_runs"].isna()]


,match_id,team_1,team_2,team_1_runs,team_2_runs,winner
46,64889,West Indies,NaN,57,NaN,NaN
123,65273,Pakistan,NaN,73,NaN,NaN
152,65652,Australia,NaN,263,NaN,NaN
194,66296,England,NaN,81,NaN,NaN
204,66356,India,NaN,46,NaN,NaN
...,...,...,...,...,...,...
2311,1395700,New Zealand,NaN,136,NaN,NaN
2315,1408106,Zimbabwe,NaN,121,NaN,NaN
2406,1450707,Nepal,NaN,26,NaN,NaN
2412,1456445,New Zealand,NaN,112,NaN,NaN


In [25]:
match_df[match_df.isna().any(axis=1)].head()

,match_id,team_1,team_2,team_1_runs,team_2_runs,winner
46,64889,West Indies,NaN,57,NaN,NaN
123,65273,Pakistan,NaN,73,NaN,NaN
152,65652,Australia,NaN,263,NaN,NaN
194,66296,England,NaN,81,NaN,NaN
204,66356,India,NaN,46,NaN,NaN


In [26]:
valid_match_df = match_df.dropna(
    subset=["team_1_runs","team_2_runs"]
).copy()
valid_match_df.shape

(2438, 6)

In [27]:
valid_match_df = valid_match_df[
    (valid_match_df["team_1_runs"] >= 0) &
    (valid_match_df["team_2_runs"] >= 0)
]

In [28]:
valid_match_df.to_csv(
    "data/processed/match_summary_valid.csv",index=False
)
print("match_summary_valid.csv saved")

match_summary_valid.csv saved


In [29]:
valid_match_ids = set(valid_match_df["match_id"])

innings_df = pd.read_csv("data/processed/innings_summary.csv")
innings_df = innings_df[
    innings_df["match_id"].isin(valid_match_ids)
]
innings_df.to_csv(
    "data/processed/innings_summary_valid.csv",index=False
)
print("innings_summary_valid.csv saved")

innings_summary_valid.csv saved


In [32]:
print(valid_match_df.isna().sum())
print(valid_match_df.shape)
#valid_match_df.head()


match_id       0
team_1         0
team_2         0
team_1_runs    0
team_2_runs    0
winner         0
dtype: int64
(2438, 6)


In [4]:
import yaml
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [6]:
yaml_dir = Path("data/raw/cricsheet/odis_yaml")

metadata = []

for file in tqdm(yaml_dir.glob("*.yaml")):
    with open(file, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)

    info = data.get("info", {})

    match_id = file.stem  # filename = match_id

    venue = info.get("venue")

    toss = info.get("toss", {})
    toss_winner = toss.get("winner")
    toss_decision = toss.get("decision")

    outcome = info.get("outcome", {})
    yaml_winner = outcome.get("winner")

    metadata.append({
        "match_id": match_id,
        "venue": venue,
        "toss_winner": toss_winner,
        "toss_decision": toss_decision,
        "yaml_winner": yaml_winner
    })


2504it [09:01,  4.62it/s]


In [7]:
metadata_df = pd.DataFrame(metadata)
metadata_df.head()


,match_id,venue,toss_winner,toss_decision,yaml_winner
0,1000887,"Brisbane Cricket Ground, Woolloongabba",Australia,bat,Australia
1,1000889,Melbourne Cricket Ground,Australia,bat,Pakistan
2,1000891,Western Australia Cricket Association Ground,Australia,field,Australia
3,1000893,Sydney Cricket Ground,Australia,bat,Australia
4,1000895,Adelaide Oval,Australia,bat,Australia


In [8]:
match_df = pd.read_csv("data/processed/match_summary_valid.csv")

match_df = match_df.merge(
    metadata_df,
    on="match_id",
    how="left"
)


ValueError: You are trying to merge on int64 and object columns for key 'match_id'. If you wish to proceed you should use pd.concat

In [10]:
match_df["match_id"].dtype,metadata_df["match_id"].dtype


(dtype('int64'), dtype('O'))

In [11]:
match_df["match_id"] = match_df["match_id"].astype(str)
metadata_df["match_id"] = metadata_df["match_id"].astype(str)


In [12]:
match_df = match_df.merge(
    metadata_df,
    on="match_id",
    how="left"
)


In [13]:
match_df.isna().sum()


match_id          0
team_1            0
team_2            0
team_1_runs       0
team_2_runs       0
winner            0
venue             0
toss_winner       0
toss_decision     0
yaml_winner      61
dtype: int64

In [14]:
match_df = match_df.drop(columns=["yaml_winner"])


In [15]:
match_df.isna().sum()

match_id         0
team_1           0
team_2           0
team_1_runs      0
team_2_runs      0
winner           0
venue            0
toss_winner      0
toss_decision    0
dtype: int64

In [16]:
match_df.to_csv(
    "data/processed/match_summary_final.csv",
    index=False
)

print("match_summary_final.csv saved")


match_summary_final.csv saved
